Today, we will use Python to analytically solve one of the most important partial differential equations out there, the diffusion equation. It is a fundamental equation that arises in many areas of physics, chemistry, biology, and engineering, and has an enormous range of applications. For instance, in physics, the diffusion equation is used to describe the movement of heat through a solid or fluid, as well as the transport of particles in gases and liquids. Also, the Schrödinger equation is (in principle) a kind of diffusion equation. In biology, the diffusion equation is used to study the movement of nutrients and other substances through living tissues, as well as the spread of diseases. In engineering, the diffusion equation is used to design and optimize processes such as heat exchangers, catalytic converters, and solar cells. It is even applied in economics, where the diffusion equation can be used to model the spread of ideas or innovations through a population. Due to its wide range of applications, it is not surprising that it was found independently by several scientists of the 19th century working on completely different topics. The famous George Gabriel Stokes derived the equation in 1845 to describe the motion of fluids, while the even more famous James Clerk Maxwell derived the equation in 1860 to describe the motion of heat.

The equation is given by:

$$ \alpha^2 \space \frac{\partial \space u}{\partial t} = \nabla ^2 u$$

which is in fact:

$$ \alpha^2 \space \frac{\partial \space u(t,x)}{\partial t} = \frac{\partial^2 \space u(t,x)}{\partial x^2}  $$

where t is time, 𝛼 a constant, and ∇² is the Laplacian operator. For concreteness, we will solve the following example application. We have a 10cm long bar with insulated sides initially at 100° everywhere. Starting at 𝑡=0, the ends are held at temperature 0°. Our task is to find the temperature distribution at some later time 𝑡.

The method we will use is the separation of variables, i.e. we use the ansatz

$$  u = X(x) \space T(t)$$

where 𝑇 and 𝑋 are functions of a single variable 𝑡 and 𝑥, respectively. Start a new Jupyter notebook and define the symbols for our problem:

In [1]:
from sympy import *

t, x, 𝛼 = symbols(r"t, x, \alpha")

u, T, X = symbols("u, T, X", cls=Function)

u = u(t, x)
T = T(t)
X = X(x)

EQ0 = Eq( 𝛼**2*diff(u,t),diff(u, (x,2)))

In [2]:
EQ0

Eq(\alpha**2*Derivative(u(t, x), t), Derivative(u(t, x), (x, 2)))

Insert the separation ansatz:

In [3]:
EQ0.subs(u, T*X).doit()

Eq(\alpha**2*X(x)*Derivative(T(t), t), T(t)*Derivative(X(x), (x, 2)))

In [4]:
Eq( _.lhs / (T*X), _.rhs / (T*X))

Eq(\alpha**2*Derivative(T(t), t)/T(t), Derivative(X(x), (x, 2))/X(x))

In [5]:
EQ1 = _

The key observation here is that the left side depends only on t, whereas the right side depends only on x. So, when you change 𝑡, only the left side changes but not the right side. Conversely, if you change 𝑥, the right side changes, but not the left side. The only way how this can be true is that both sides are equal to a constant. Let’s call that constant −𝑘²:

In [6]:
k = Symbol("k")

Eq( EQ1.lhs, -k**2 )

Eq(\alpha**2*Derivative(T(t), t)/T(t), -k**2)

This is just an ordinary differential equation that we can easily solve:

In [7]:
k = Symbol("k")

Eq( EQ1.lhs, -k**2 )

Eq(T(t), C1*exp(-k**2*t/\alpha**2))

Now focus on the side of the separated equation:

In [8]:
Eq( EQ1.rhs, -k**2 )

Eq(Derivative(X(x), (x, 2))/X(x), -k**2)

Again, a simple ODE:

In [9]:
sol_X = dsolve(_, X)
sol_X

Eq(X(x), C1*exp(-I*k*x) + C2*exp(I*k*x))

Hmmm… sympy uses the same names of the constants of integration, so we get a name clash. Let’s rename the constants:

In [10]:
C1, C2, A, B, C = symbols("C1, C2, A, B, C")

sol_T = sol_T.subs(C1, C)
sol_X = sol_X.subs({C1: A, C2: B})

In [11]:
sol_T

Eq(T(t), C*exp(-k**2*t/\alpha**2))

In [12]:
sol_X

Eq(X(x), A*exp(-I*k*x) + B*exp(I*k*x))

Since the exponentials are complex, it’s more convenient to have that in terms of sines and cosines, so we can rewrite that:

In [13]:
sol_X.rhs.rewrite(sin)

A*(-I*sin(k*x) + cos(k*x)) + B*(I*sin(k*x) + cos(k*x))

In [14]:
_.expand()

-I*A*sin(k*x) + A*cos(k*x) + I*B*sin(k*x) + B*cos(k*x)

In [15]:
collect(_,[sin(k*x), cos(k*x)])

(A + B)*cos(k*x) + (-I*A + I*B)*sin(k*x)

The coefficients are independent, so we can simply give new names to them:

In [16]:
_.replace(A + B, B).replace(-I*A + I*B, A)

A*sin(k*x) + B*cos(k*x)

In [17]:
sol_X = Eq(X, _)
sol_X

Eq(X(x), A*sin(k*x) + B*cos(k*x))

And that’s the form we will work with.

Now consider the first boundary conditions at 𝑥=0. This gives a value for the constant 𝐵:

In [18]:
Eq( sol_X.rhs.subs(x, 0), 0)

Eq(B, 0)

In [19]:
sol_X = sol_X.subs(B, 0)
sol_X

Eq(X(x), A*sin(k*x))

The other boundary condition gives a constraint on 𝑘:

In [20]:
L = Symbol("L", real=True, positive=True)

Eq( sol_X.rhs.subs(x,L), 0)

Eq(A*sin(L*k), 0)

In [21]:
n = Symbol("n", Integer=True, positive=True)

Eq( L*k, n*pi )

Eq(L*k, pi*n)

In [22]:
solve(_, k)

[pi*n/L]

In [23]:
sol_X = sol_X.subs(k, _[0] )
sol_X

Eq(X(x), A*sin(pi*n*x/L))

Now we can assemble a solution from our base solutions.

In [24]:
a = IndexedBase("a")

sol_u = Eq( u, Sum( sol_T.rhs.subs({C:1, k: __[0]}) * sol_X.rhs.subs(A, a[n]), (n, 1, oo)) )
sol_u

Eq(u(t, x), Sum(exp(-pi**2*n**2*t/(L**2*\alpha**2))*sin(pi*n*x/L)*a[n], (n, 1, oo)))

where we have absorbed the coefficient 𝐶 into the new coefficients 𝑎_𝑛.

But this must still satisfy the initial condition, which we haven’t evaluated yet. So do that now:

In [25]:
_.subs(t,0)

Eq(u(0, x), Sum(sin(pi*n*x/L)*a[n], (n, 1, oo)))

In [26]:
Eq( _.rhs,100 )

Eq(Sum(sin(pi*n*x/L)*a[n], (n, 1, oo)), 100)

We can solve for 𝑎_𝑛 by using the fact that the sines are orthogonal. So multiply both sides by a suitable sine and integrate:

In [27]:
m = Symbol("m", integer=True, positive=True)

Eq( Sum( Integral(a[n]*sin(pi*n*x/L)*sin(pi*m*x/L), (x, 0, L)),(n, 1, oo)), Integral(100*sin(pi*m*x/L), (x, 0, L)))

Eq(Sum(Integral(sin(pi*m*x/L)*sin(pi*n*x/L)*a[n], (x, 0, L)), (n, 1, oo)), Integral(100*sin(pi*m*x/L), (x, 0, L)))

### THis is where the problem start!

It does not simplify the $ sin(\pi n) $ in the case of $ m \neq n$ and the $ sin^2(\pi n) $ and $ cos^2(\pi n) $.

In [28]:
_.doit()

Eq(Sum(Piecewise((-(-1)**m*L*m*sin(pi*n)*a[n]/(pi*m**2 - pi*n**2), Ne(m, n)), ((L*sin(pi*n)**2/2 + L*cos(pi*n)**2/2 - L*sin(pi*n)*cos(pi*n)/(2*pi*n))*a[n], True)), (n, 1, oo)), -100*(-1)**m*L/(pi*m) + 100*L/(pi*m))

### And here is the version of sympy

In [30]:
import sympy
sympy.__version__

'1.11.1'

### The version of python is 3.10.10

## The rest is for later.

The infinite sum on the left side has only one non-zero term, for which 𝑚=𝑛:

In [ ]:
solve(_, a[m])

In [ ]:
Eq(a[m], _[0])

Now insert the values for the coefficients in our solution:

In [ ]:
sol_u = sol_u.replace(a[n], _.rhs.subs(m,n))
sol_u

And that is the exact solution to our problem.

Finally, we can plot it. But of course, we have to cut off the series at some point. We will approximate the solution by the first 30 terms:

In [ ]:
sol_u_approx = Sum(sol_u.rhs.args[0], (n, 1, 30)).subs([L: 10,alpha: 1})
sol_u_approx

Now we can plot the solution with the sympy plotting backends spb:


In [ ]:
from spb import *
plot_contour(sol_u_approx, (x, 0, 10) , (t, 0, 8 ));